Creation of raw MFCC data from audio files.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
import os

folder = '/content/drive/My Drive/Colab Notebooks/COVID-19'
if not os.path.exists(folder):
  print(folder + ' does not exist')

mfcc_folder = os.path.join(folder, 'mfcc_json_files')
if not os.path.exists(mfcc_folder):
  print(mfcc_folder + ' does not exist')

In [3]:
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import os
import librosa
import librosa.display
import cv2
import numpy as np
import json
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore")

/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


Read in dataframes created previously.

In [5]:
df_train = pd.read_csv(os.path.join(folder, 'df_train.csv'))
print(df_train.shape)
df_train.head(10)

(11401, 9)


,Unnamed: 0,id,age,gender,status,audio_file,image_file,source,cough_detected
0,0,Zb2Inq1UVhcMnWA41GpKyRdF4z62,66.0,female,healthy,/content/drive/My Drive/Colab Notebooks/coswar...,/content/drive/My Drive/Colab Notebooks/coswar...,coswara,NaN
1,1,c4138919-bd2c-4e24-9fcf-582e428c7d27,37.0,female,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,/content/drive/My Drive/Colab Notebooks/coughv...,coughvid,1.0000
2,2,ygiPwZYsKFetOejhxWXB6rKSHAf2,66.0,male,COVID-19,/content/drive/My Drive/Colab Notebooks/coswar...,/content/drive/My Drive/Colab Notebooks/coswar...,coswara,NaN
3,3,ec8ddd1e-d555-490f-9719-70a99c1c6993,29.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,/content/drive/My Drive/Colab Notebooks/coughv...,coughvid,0.7849
4,4,CFkIEzwZ5PcogCElOD2K3RJQzmh1,44.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coswar...,/content/drive/My Drive/Colab Notebooks/coswar...,coswara,NaN
5,5,68b505f9-1bf6-4a96-be6a-6cebfc73a974,NaN,female,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,/content/drive/My Drive/Colab Notebooks/coughv...,coughvid,0.9859
6,6,b22dc80a-2eec-4b9f-a69b-7eb5353a08e8,28.0,female,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,/content/drive/My Drive/Colab Notebooks/coughv...,coughvid,0.8467
7,7,428c81f6-943c-4679-a034-ec004e26d040,50.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,/content/drive/My Drive/Colab Notebooks/coughv...,coughvid,0.9867
8,8,17406b1b-fe8e-4325-bbf2-3a0234e65f2b,54.0,female,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,/content/drive/My Drive/Colab Notebooks/coughv...,coughvid,0.6240
9,9,587204a9-b8d4-4f10-b70c-cbf2db847af1,50.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,/content/drive/My Drive/Colab Notebooks/coughv...,coughvid,0.8438


In [6]:
df_test = pd.read_csv(os.path.join(folder, 'df_test.csv'))
print(df_test.shape)
df_test.head(10)

(1000, 9)


,Unnamed: 0,id,age,gender,status,audio_file,image_file,source,cough_detected
0,0,y6VWQQ5bW0drHBBQ74CmfwHniKo2,37.0,female,COVID-19,/content/drive/My Drive/Colab Notebooks/coswar...,/content/drive/My Drive/Colab Notebooks/coswar...,coswara,NaN
1,1,e614bfdf-0264-4094-8585-a1cd4068f503,22.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,/content/drive/My Drive/Colab Notebooks/coughv...,coughvid,1.0000
2,2,b4b8c8d8-37cf-48e5-b5af-12009c442239,30.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,/content/drive/My Drive/Colab Notebooks/coughv...,coughvid,0.9887
3,3,8b04c1fa-4fbe-45f3-9b4a-5cc6a450c43c,32.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,/content/drive/My Drive/Colab Notebooks/coughv...,coughvid,1.0000
4,4,b24347b4-ab14-46c7-9863-c62cd5dd0659,28.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,/content/drive/My Drive/Colab Notebooks/coughv...,coughvid,0.9650
5,5,DqztVX8gWrOi4il3xouSTV4FwyC3,43.0,male,COVID-19,/content/drive/My Drive/Colab Notebooks/coswar...,/content/drive/My Drive/Colab Notebooks/coswar...,coswara,NaN
6,6,466aPruIIbOEG3V1IaKzu5O8kun2,41.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coswar...,/content/drive/My Drive/Colab Notebooks/coswar...,coswara,NaN
7,7,ce3f9e37-b553-4c23-b341-8da601f7468e,23.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,/content/drive/My Drive/Colab Notebooks/coughv...,coughvid,1.0000
8,8,843d0600-495e-441c-88f9-18695bf67b24,40.0,male,healthy,/content/drive/My Drive/Colab Notebooks/coughv...,/content/drive/My Drive/Colab Notebooks/coughv...,coughvid,0.8022
9,9,9c9d5f33-e8e4-4483-a9ba-992fe633e486,20.0,female,COVID-19,/content/drive/My Drive/Colab Notebooks/coughv...,/content/drive/My Drive/Colab Notebooks/coughv...,coughvid,1.0000


Use code from virufy.com to extract features from audio files.

(I have made changes to reflect my variable names, 
stored the results in a simpler directory format, 
kept all the MFCCs going through the sample rather than taking the mean, and I have not split into test and train yet which they had done with their data.)

In [20]:
# Functions to process audio files into images (adapted from code at virufy.com)
def trim_silence(x, *args):
    try:pad,db_max,frame_length,hop_length = args[0],args[1],args[2],args[3]
    except: 
        print('Please enter the following arguments: pad,db_max,frame_length,hop_length')
        return

    _, ints = librosa.effects.trim(x, top_db=db_max, frame_length=256, hop_length=64)
    start = int(max(ints[0]-pad, 0))
    end   = int(min(ints[1]+pad, len(x)))
    return x[start:end]

def process_cough_file(path,trim,*args):
    try: sr,removeaudio,chunk,db_max = args[0],args[1],args[2],args[3]
    except: 
        sr,removeaudio,chunk,db_max= 48000,False,3,50
    try:
        x,sr = librosa.load(path, sr=sr)       
    except: 
        return -1
    
    if len(x)/sr < 0.3 or len(x)/sr > 30:
        return None,None  
    hop_length = np.floor(0.010*sr).astype(int) #10ms
    win_length = np.floor(0.020*sr).astype(int) #20ms  

    if removeaudio:
        os.remove(path)
    
    x = trim(x, 0.25*sr, db_max,win_length,hop_length) 
    x = x[:np.floor(chunk*sr).astype(int)]
    
    #pads to chunk size if smaller
    x_pad = np.zeros(int(sr*chunk))
    x_pad[:min(len(x_pad), len(x))] = x[:min(len(x_pad), len(x))]

    return [x_pad,sr,hop_length,win_length]

def get_rawMFCCs(audio,sr,*args):
    try: hop_length,win_length,n_mfcc,n_mels,n_ftt = args[0],args[1],args[2],args[3],args[4]
    except:
        hop_length = np.floor(0.010*sr).astype(int) #10ms
        win_length = np.floor(0.020*sr).astype(int) #20ms  
        n_mfcc,n_mels,n_ftt=13,13,2048

    rawMFCCs    = librosa.feature.mfcc(y=audio,sr=sr, n_mfcc=n_mfcc,n_mels=n_mels, n_fft=n_ftt, hop_length=hop_length)
    rawMFCCs    = rawMFCCs.T

    return rawMFCCs

def getlabel(key, dataframe, chosen):
      return dataframe.loc[dataframe[chosen['id']]==key][chosen['status']].tolist()[0]

def extract(df, chosen): 

    # dictionary to store labels, and MFCCs
    data = {
        "labels": [],
        "mfcc": []
    }
        
    keys, dirs = df[chosen['id']].tolist(),df[chosen['path']].tolist()  
    audio_objs = [process_cough_file(path,trim_silence) for path in dirs]
    false_indices = [i for i in range(len(audio_objs)) if isinstance(audio_objs[i],int) or isinstance(audio_objs[i],tuple)]

    audio_objs = [audio_objs[i] for i in range(len(audio_objs)) if i not in false_indices]
    audio_objs = np.array(audio_objs)
    audio,sr,hop_length,win_length = audio_objs[:,0],audio_objs[:,1],audio_objs[:,2],audio_objs[:,3]
    
    dirs = [dirs[i] for i in range(len(dirs)) if i not in false_indices]
    keys = [keys[i] for i in range(len(keys)) if i not in false_indices]

    j=0 #This is just so I can see progress
    for key,a_i,sr_i in list(zip(keys,audio,sr)):
      rawMFCC = get_rawMFCCs(a_i,sr_i)
      label = getlabel(key, df, chosen)
      data["mfcc"].append(rawMFCC.tolist())
      data["labels"].append(label)
      j += 1
      print("Files processed:{}".format(j))

    return data

def filter_DF(df):
    names = list(df.columns)
    chosen= {}
    for name in names:
        if 'status' in name.lower():chosen['status'] = name # Choosing the target 
        elif 'audio_file' in name.lower():chosen['path'] = name
        elif 'uuid' in name.lower() or 'id' == name.lower() :chosen['id'] = name
    return df[[chosen['id'],chosen['status'],chosen['path']]].dropna().reset_index(), chosen 

def extract_features(df):
    dataframe, chosen = filter_DF(df)
    features = extract(dataframe, chosen) #, (folder + '/coughvid_images/'))
    return features

Test for a couple of files to check size, etc.

In [19]:
features1 = extract_features(df_train.iloc[:2])
features1

Files processed:2
Files processed:3


{'labels': ['healthy', 'healthy'],
 'mfcc': [[[-195.88192639591887,
    0.0,
    -5.411986781997531e-15,
    0.0,
    4.935492756485594e-15,
    0.0,
    0.0,
    0.0,
    0.0,
    2.590346423698442e-15,
    0.0,
    -1.33393493678406e-15,
    0.0],
   [-195.88192639591887,
    0.0,
    -5.411986781997531e-15,
    0.0,
    4.935492756485594e-15,
    0.0,
    0.0,
    0.0,
    0.0,
    2.590346423698442e-15,
    0.0,
    -1.33393493678406e-15,
    0.0],
   [-195.76011650134387,
    0.1710091976414843,
    0.16725949116295286,
    0.16107076466770312,
    0.15253326372382794,
    0.14177148431984515,
    0.12894235743290625,
    0.11423296062169327,
    0.09785779001439066,
    0.08005563247212968,
    0.061086083538831604,
    0.04122576195366057,
    0.020764275926876546],
   [-195.6732162411782,
    0.2930086773850204,
    0.28658389701641634,
    0.2759800780986053,
    0.26135184818909674,
    0.24291251981987236,
    0.22093097992032162,
    0.1957277688402922,
    0.16767040615001

In [28]:
print(len(features1['mfcc']),len(features1['mfcc'][0]),len(features1['mfcc'][0][0]),len(features1['mfcc'][1]),len(features1['mfcc'][1][0]))
print(features1['labels'])

2 301 13 301 13
['healthy', 'healthy']


Extracting the mfccs from the audio files.
Google crashes with too many files at once so I have processed them in batches of 1000.

In [29]:
test_features = extract_features(df_test)

Files processed:1
Files processed:2
Files processed:3
Files processed:4
Files processed:5
Files processed:6
Files processed:7
Files processed:8
Files processed:9
Files processed:10
Files processed:11
Files processed:12
Files processed:13
Files processed:14
Files processed:15
Files processed:16
Files processed:17
Files processed:18
Files processed:19
Files processed:20
Files processed:21
Files processed:22
Files processed:23
Files processed:24
Files processed:25
Files processed:26
Files processed:27
Files processed:28
Files processed:29
Files processed:30
Files processed:31
Files processed:32
Files processed:33
Files processed:34
Files processed:35
Files processed:36
Files processed:37
Files processed:38
Files processed:39
Files processed:40
Files processed:41
Files processed:42
Files processed:43
Files processed:44
Files processed:45
Files processed:46
Files processed:47
Files processed:48
Files processed:49
Files processed:50
Files processed:51
Files processed:52
Files processed:53
Fi

In [30]:
#Save dictionary of features to save processing again as it takes a long time.
with open(os.path.join(mfcc_folder, 'test_mfccs.json'), 'w') as f:
    json.dump(test_features, f, indent=4)

In [31]:
features2 = extract_features(df_train.iloc[2:1000]) # The first 2 rows are extracted earlier in notebook - features1 

Files processed:1
Files processed:2
Files processed:3
Files processed:4
Files processed:5
Files processed:6
Files processed:7
Files processed:8
Files processed:9
Files processed:10
Files processed:11
Files processed:12
Files processed:13
Files processed:14
Files processed:15
Files processed:16
Files processed:17
Files processed:18
Files processed:19
Files processed:20
Files processed:21
Files processed:22
Files processed:23
Files processed:24
Files processed:25
Files processed:26
Files processed:27
Files processed:28
Files processed:29
Files processed:30
Files processed:31
Files processed:32
Files processed:33
Files processed:34
Files processed:35
Files processed:36
Files processed:37
Files processed:38
Files processed:39
Files processed:40
Files processed:41
Files processed:42
Files processed:43
Files processed:44
Files processed:45
Files processed:46
Files processed:47
Files processed:48
Files processed:49
Files processed:50
Files processed:51
Files processed:52
Files processed:53
Fi

In [32]:
features3 = extract_features(df_train.iloc[1000:2000])

Files processed:1
Files processed:2
Files processed:3
Files processed:4
Files processed:5
Files processed:6
Files processed:7
Files processed:8
Files processed:9
Files processed:10
Files processed:11
Files processed:12
Files processed:13
Files processed:14
Files processed:15
Files processed:16
Files processed:17
Files processed:18
Files processed:19
Files processed:20
Files processed:21
Files processed:22
Files processed:23
Files processed:24
Files processed:25
Files processed:26
Files processed:27
Files processed:28
Files processed:29
Files processed:30
Files processed:31
Files processed:32
Files processed:33
Files processed:34
Files processed:35
Files processed:36
Files processed:37
Files processed:38
Files processed:39
Files processed:40
Files processed:41
Files processed:42
Files processed:43
Files processed:44
Files processed:45
Files processed:46
Files processed:47
Files processed:48
Files processed:49
Files processed:50
Files processed:51
Files processed:52
Files processed:53
Fi

In [35]:
features4 = extract_features(df_train.iloc[2000:3000])

Files processed:1
Files processed:2
Files processed:3
Files processed:4
Files processed:5
Files processed:6
Files processed:7
Files processed:8
Files processed:9
Files processed:10
Files processed:11
Files processed:12
Files processed:13
Files processed:14
Files processed:15
Files processed:16
Files processed:17
Files processed:18
Files processed:19
Files processed:20
Files processed:21
Files processed:22
Files processed:23
Files processed:24
Files processed:25
Files processed:26
Files processed:27
Files processed:28
Files processed:29
Files processed:30
Files processed:31
Files processed:32
Files processed:33
Files processed:34
Files processed:35
Files processed:36
Files processed:37
Files processed:38
Files processed:39
Files processed:40
Files processed:41
Files processed:42
Files processed:43
Files processed:44
Files processed:45
Files processed:46
Files processed:47
Files processed:48
Files processed:49
Files processed:50
Files processed:51
Files processed:52
Files processed:53
Fi

In [36]:
features5 = extract_features(df_train.iloc[3000:4000])

Files processed:1
Files processed:2
Files processed:3
Files processed:4
Files processed:5
Files processed:6
Files processed:7
Files processed:8
Files processed:9
Files processed:10
Files processed:11
Files processed:12
Files processed:13
Files processed:14
Files processed:15
Files processed:16
Files processed:17
Files processed:18
Files processed:19
Files processed:20
Files processed:21
Files processed:22
Files processed:23
Files processed:24
Files processed:25
Files processed:26
Files processed:27
Files processed:28
Files processed:29
Files processed:30
Files processed:31
Files processed:32
Files processed:33
Files processed:34
Files processed:35
Files processed:36
Files processed:37
Files processed:38
Files processed:39
Files processed:40
Files processed:41
Files processed:42
Files processed:43
Files processed:44
Files processed:45
Files processed:46
Files processed:47
Files processed:48
Files processed:49
Files processed:50
Files processed:51
Files processed:52
Files processed:53
Fi

In [37]:
features6 = extract_features(df_train.iloc[4000:5000])

Files processed:1
Files processed:2
Files processed:3
Files processed:4
Files processed:5
Files processed:6
Files processed:7
Files processed:8
Files processed:9
Files processed:10
Files processed:11
Files processed:12
Files processed:13
Files processed:14
Files processed:15
Files processed:16
Files processed:17
Files processed:18
Files processed:19
Files processed:20
Files processed:21
Files processed:22
Files processed:23
Files processed:24
Files processed:25
Files processed:26
Files processed:27
Files processed:28
Files processed:29
Files processed:30
Files processed:31
Files processed:32
Files processed:33
Files processed:34
Files processed:35
Files processed:36
Files processed:37
Files processed:38
Files processed:39
Files processed:40
Files processed:41
Files processed:42
Files processed:43
Files processed:44
Files processed:45
Files processed:46
Files processed:47
Files processed:48
Files processed:49
Files processed:50
Files processed:51
Files processed:52
Files processed:53
Fi

In [38]:
features7 = extract_features(df_train.iloc[5000:6000])

Files processed:1
Files processed:2
Files processed:3
Files processed:4
Files processed:5
Files processed:6
Files processed:7
Files processed:8
Files processed:9
Files processed:10
Files processed:11
Files processed:12
Files processed:13
Files processed:14
Files processed:15
Files processed:16
Files processed:17
Files processed:18
Files processed:19
Files processed:20
Files processed:21
Files processed:22
Files processed:23
Files processed:24
Files processed:25
Files processed:26
Files processed:27
Files processed:28
Files processed:29
Files processed:30
Files processed:31
Files processed:32
Files processed:33
Files processed:34
Files processed:35
Files processed:36
Files processed:37
Files processed:38
Files processed:39
Files processed:40
Files processed:41
Files processed:42
Files processed:43
Files processed:44
Files processed:45
Files processed:46
Files processed:47
Files processed:48
Files processed:49
Files processed:50
Files processed:51
Files processed:52
Files processed:53
Fi

In [39]:
features8 = extract_features(df_train.iloc[6000:7000])

Files processed:1
Files processed:2
Files processed:3
Files processed:4
Files processed:5
Files processed:6
Files processed:7
Files processed:8
Files processed:9
Files processed:10
Files processed:11
Files processed:12
Files processed:13
Files processed:14
Files processed:15
Files processed:16
Files processed:17
Files processed:18
Files processed:19
Files processed:20
Files processed:21
Files processed:22
Files processed:23
Files processed:24
Files processed:25
Files processed:26
Files processed:27
Files processed:28
Files processed:29
Files processed:30
Files processed:31
Files processed:32
Files processed:33
Files processed:34
Files processed:35
Files processed:36
Files processed:37
Files processed:38
Files processed:39
Files processed:40
Files processed:41
Files processed:42
Files processed:43
Files processed:44
Files processed:45
Files processed:46
Files processed:47
Files processed:48
Files processed:49
Files processed:50
Files processed:51
Files processed:52
Files processed:53
Fi

In [40]:
features9 = extract_features(df_train.iloc[7000:8000])

Files processed:1
Files processed:2
Files processed:3
Files processed:4
Files processed:5
Files processed:6
Files processed:7
Files processed:8
Files processed:9
Files processed:10
Files processed:11
Files processed:12
Files processed:13
Files processed:14
Files processed:15
Files processed:16
Files processed:17
Files processed:18
Files processed:19
Files processed:20
Files processed:21
Files processed:22
Files processed:23
Files processed:24
Files processed:25
Files processed:26
Files processed:27
Files processed:28
Files processed:29
Files processed:30
Files processed:31
Files processed:32
Files processed:33
Files processed:34
Files processed:35
Files processed:36
Files processed:37
Files processed:38
Files processed:39
Files processed:40
Files processed:41
Files processed:42
Files processed:43
Files processed:44
Files processed:45
Files processed:46
Files processed:47
Files processed:48
Files processed:49
Files processed:50
Files processed:51
Files processed:52
Files processed:53
Fi

In [41]:
features10 = extract_features(df_train.iloc[8000:9000])

Files processed:1
Files processed:2
Files processed:3
Files processed:4
Files processed:5
Files processed:6
Files processed:7
Files processed:8
Files processed:9
Files processed:10
Files processed:11
Files processed:12
Files processed:13
Files processed:14
Files processed:15
Files processed:16
Files processed:17
Files processed:18
Files processed:19
Files processed:20
Files processed:21
Files processed:22
Files processed:23
Files processed:24
Files processed:25
Files processed:26
Files processed:27
Files processed:28
Files processed:29
Files processed:30
Files processed:31
Files processed:32
Files processed:33
Files processed:34
Files processed:35
Files processed:36
Files processed:37
Files processed:38
Files processed:39
Files processed:40
Files processed:41
Files processed:42
Files processed:43
Files processed:44
Files processed:45
Files processed:46
Files processed:47
Files processed:48
Files processed:49
Files processed:50
Files processed:51
Files processed:52
Files processed:53
Fi

In [42]:
features11 = extract_features(df_train.iloc[9000:10000])

Files processed:1
Files processed:2
Files processed:3
Files processed:4
Files processed:5
Files processed:6
Files processed:7
Files processed:8
Files processed:9
Files processed:10
Files processed:11
Files processed:12
Files processed:13
Files processed:14
Files processed:15
Files processed:16
Files processed:17
Files processed:18
Files processed:19
Files processed:20
Files processed:21
Files processed:22
Files processed:23
Files processed:24
Files processed:25
Files processed:26
Files processed:27
Files processed:28
Files processed:29
Files processed:30
Files processed:31
Files processed:32
Files processed:33
Files processed:34
Files processed:35
Files processed:36
Files processed:37
Files processed:38
Files processed:39
Files processed:40
Files processed:41
Files processed:42
Files processed:43
Files processed:44
Files processed:45
Files processed:46
Files processed:47
Files processed:48
Files processed:49
Files processed:50
Files processed:51
Files processed:52
Files processed:53
Fi

In [43]:
features12 = extract_features(df_train.iloc[10000:])

Files processed:1
Files processed:2
Files processed:3
Files processed:4
Files processed:5
Files processed:6
Files processed:7
Files processed:8
Files processed:9
Files processed:10
Files processed:11
Files processed:12
Files processed:13
Files processed:14
Files processed:15
Files processed:16
Files processed:17
Files processed:18
Files processed:19
Files processed:20
Files processed:21
Files processed:22
Files processed:23
Files processed:24
Files processed:25
Files processed:26
Files processed:27
Files processed:28
Files processed:29
Files processed:30
Files processed:31
Files processed:32
Files processed:33
Files processed:34
Files processed:35
Files processed:36
Files processed:37
Files processed:38
Files processed:39
Files processed:40
Files processed:41
Files processed:42
Files processed:43
Files processed:44
Files processed:45
Files processed:46
Files processed:47
Files processed:48
Files processed:49
Files processed:50
Files processed:51
Files processed:52
Files processed:53
Fi

In [68]:
audio_data = features1['mfcc']  + features2['mfcc'] + features3['mfcc'] + features4['mfcc'] + \
        features5['mfcc'] + features6['mfcc'] + features7['mfcc'] + features8['mfcc'] + \
        features9['mfcc'] + features10['mfcc'] + features11['mfcc'] + features12['mfcc']
label_data = features1['labels']  + features2['labels'] + features3['labels'] + features4['labels'] + \
        features5['labels'] + features6['labels'] + features7['labels'] + features8['labels'] + \
        features9['labels'] + features10['labels'] + features11['labels'] + features12['labels']

In [69]:
train_features = {
        "labels": label_data,
        "mfcc": audio_data
    }

In [71]:
#Save dictionary of features to save processing again as it takes a long time.
with open(os.path.join(mfcc_folder, 'train_mfccs.json'), 'w') as f:
    json.dump(train_features, f, indent=4)

In [46]:
X = np.array(test_features["mfcc"])
X.shape

(1000, 301, 13)

In [70]:
X = np.array(train_features["mfcc"])
X.shape

(11401, 301, 13)

In [72]:
X = np.array(test_features["labels"])
X.shape

(1000,)

In [73]:
X = np.array(train_features["labels"])
X.shape

(11401,)

Test loading

In [74]:
with open(os.path.join(mfcc_folder, 'train_mfccs.json')) as json_file:
    check = json.load(json_file)

X = np.array(check["mfcc"])
X.shape

(11401, 301, 13)